In [26]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import pandas as pd
import os
import reionizer
import astropy
import astropy.cosmology
from astropy.cosmology import Planck15 as P15
from scipy import constants

#Call data files
LFz_data_dir = os.environ['LYA_DATA_DIR']+'data/models/' #calls directory
LFz_dir = LFz_data_dir+'MTT15_UVLF/LF_pred/' #inside models folder call MTT15_UVLF/LF_pred/ folder
LFz_files = sorted(reionizer.insensitive_glob(LFz_dir+'LF_pred_z*.txt')) 
#calls each file in modelled data * will be replaced with corresponding zval


In [18]:
#Call LF_pred files by calling file name at specific z value 
LFz_tab = np.array([float(f.split('LF_pred_z')[-1].split('.txt')[0]) for f in LFz_files]) 

#Function used to load files
def load_uvf_pandas(ufl_filename): 
    """
    Load table into pandas df
    """
    uvf_tab_df = pd.read_csv(ufl_filename, skiprows=1, delim_whitespace=True)
    
    # Shuffle the column names to remove the '#' from the first column
    uvf_tab_df.columns = np.roll(uvf_tab_df.columns, -1)

    # Cut off the last (empty) column
    uvf_tab_df = uvf_tab_df.iloc[:, :-1]

    return uvf_tab_df

In [49]:
zval_test = np.array([5.9,6.8]) #chosen redshifts
beta = -2.0 #usually -2 for high z galaxies as per spectrum as power law
pc_10 = 1e-6 #1 pc to Mpc
wl_lya = 1216 #angstrom
wl_uv = 1500 #angstrom
f0 = 3.631e-23 #flux_0 in ergs^-1 cm^-2 Hz^-1
c = constants.c #speed of light


for zz,zval2 in enumerate (LFz_tab): 
    if zval2 in zval_test:
        LF_tab = load_uvf_pandas(LFz_files[zz]) #load in files
        Muv = LF_tab['Muv'] #Muv values in UVLF data
        
        def muv(Muv,d_l,zval2): #define apparent magnitude equation
            d_l = P15.luminosity_distance(zval2).value #define distance modulus
            p1 = Muv
            p2 = 5*(np.log10(d_l/pc_10))
            p3 = 2.5*(beta+1.0)*np.log10(zval2+1.0)
            ans = p1 + p2 + p3
            return ans
        muv = muv(Muv,d_l,zval2)
        muv_rounded = round(muv,1)
        
        #flux density of UV continuum at Lya wavelength from Muv for given zvals
        fd_uv = f0 * (10**(-0.4*muv)) *(c/(wl_lya**2)) *((wl_lya/wl_uv)**(beta+2.0))
        
        #jacobian - partial EW / partial Lya Luminosity for given zvals 
        jacobian = 1/((4*np.pi*d_l**2.)*fd_uv)
        print(muv_rounded)
        
        
        

0      26.8
1      27.4
2      27.5
3      27.5
4      27.5
       ... 
176    43.6
177    43.6
178    43.6
179    43.4
180    41.7
Name: Muv, Length: 181, dtype: float64
0      26.8
1      27.4
2      27.4
3      27.5
4      27.5
       ... 
176    43.6
177    43.6
178    43.5
179    43.3
180    41.9
Name: Muv, Length: 181, dtype: float64
